In [1]:
from common.imports import *
from common.util import *
from common.const import *
from data.dataloader import *
from model.transformer import *
from model.run_model import *

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
torch.cuda.empty_cache()

cuda


In [3]:
# PARAMS


In [4]:
# OBJECTS

## MODEL
model = stonks_transformer_model(
    d_model=D_MODEL,
    nhead=NUM_HEADS,
    num_encoder_layers=NUM_ENCODER_LAYERS,
    num_decoder_layers=NUM_DECODER_LAYERS,
    dropout=DROPOUT
).to(device)
checkpoint = torch.load("best_model.pt")
model.load_state_dict(checkpoint['state_dict'])

## DATASET
dataset=ForecasterDataset(0, None)

/home/kanesweet/anaconda3/envs/stonksEnv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:218: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because num_head is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [5]:
# CHOOSE TICKER + DAYS_PRED
ticker = "TSLA"
days_pred = 5

In [6]:

idx = HISTORY_DATA().ticker_stoi(ticker)
src = dataset[idx].unsqueeze(0)[:, :-days_pred, :]
inference = inference_stonks_transformer(model, days_pred, src, device)

ground_truth = HISTORY_DATA().array_to_dataframe(array=src[0,-days_pred:,:5].numpy())
inference = HISTORY_DATA().array_to_dataframe(array=inference[0].numpy())

print("ground truth:\n",ground_truth)
print("====================================================================")
print("inference:\n",inference)



src: torch.Size([1, 3260, 5]) 
 tensor([[[1.9111, 2.5945, 1.8554, 2.5631, 1.3473],
         [2.5550, 1.9065, 1.8789, 2.5873, 1.2970],
         [2.4508, 1.0674, 2.0330, 2.6048, 1.7751],
         [1.5050, 0.8008, 2.0737, 2.6933, 1.5160],
         [1.0100, 1.5732, 2.3125, 2.7720, 1.9523]]], device='cuda:0') 
____________________________
out: torch.Size([1, 1, 5]) 
 tensor([[[-0.2011, -0.0826, -0.2140, -0.1784,  0.0564]]], device='cuda:0') 
____________________________
src: torch.Size([1, 3261, 5]) 
 tensor([[[ 2.5550,  1.9065,  1.8789,  2.5873,  1.2970],
         [ 2.4508,  1.0674,  2.0330,  2.6048,  1.7751],
         [ 1.5050,  0.8008,  2.0737,  2.6933,  1.5160],
         [ 1.0100,  1.5732,  2.3125,  2.7720,  1.9523],
         [-1.0277,  0.4801,  0.2823,  0.6897,  0.7309]]], device='cuda:0',
       grad_fn=<SliceBackward0>) 
____________________________
out: torch.Size([1, 1, 5]) 
 tensor([[[-0.2011, -0.0826, -0.2140, -0.1784,  0.0564]]], device='cuda:0') 
____________________________
sr

In [7]:
# TODO add some helpful data analysis and vizualization methods